# Analyzing hate crimes trends for Austin against the USA as a whole, 2017 - Present

## Data Wrangling & Cleaning

I've been working, off and on, on this project for since about January 2020. One-half practice, one-half because I want to try and contribute to making sense of the chaos that is our world right now. What I intend is to analyze hate crimes trends for Austin, TX against the USA as a whole from 2017 to the present, with particular focus on the LGBT Community. 

I am using data provided by Austin PD in this notebook, and in the next 2, or 3 notebooks as well. For now, I am focusing solely on data for Austin. I will get into broader data for the USA later down the road.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Let's load some data & get to work! I am utilizing data from data.austintexas.gov located in the Austin PD's database on reported hatecrimes. 
aus_17 = pd.read_csv('https://data.austintexas.gov/resource/79qh-wdpx.csv')

display(aus_17.head())

,month,incident_number,date_of_incident_day_of_week,number_of_vitims_under_18,number_of_victims_over_18,number_of_offenders_under_18,number_of_offenders_over_18,race_or_ethnic_of_offender,offense,offense_location,bias,victim_type
0,January,2017-241137,01/01/2017/Sun,0,1,0,1,White/Not Hispanic,Aggravated Assault,Park/Playground,Anti-Black or African American,Individual
1,February,2017-580344,02/01/2017/Wed,0,1,0,1,Black or African American/Not Hispanic,Aggravated Assault,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual
2,March,2017-800291,03/21/2017/Tues,0,0,0,0,Unknown,Destruction,Highway/Road/Alley/Street/Sidewalk,Anti-Jewish,Other
3,April,2017-1021534,04/12/2017/Wed,0,0,0,0,White/Unknown,Simple Assault,Air/Bus/Train Terminal,Anti-Jewish,Individual
4,May,2017-1351550,05/15/2017/Mon,1,0,1,2,White/Not Hispanic,Simple Assault,Residence/Home,Anti-Gay (Male),Individual


### First glance...
As I stated previously, my goal is to analyze trends over time. In particular, I want to focus on how hate crime affects the LGBT community. Initially speaking, most of these columns will be unnecessary for my purposes so I suspect we'll be removing most of them. 

In [3]:
# Loading the datasets for '18, '19, and this year
aus_18 = pd.read_csv('https://data.austintexas.gov/resource/idj2-d9th.csv')
aus_19 = pd.read_csv('https://data.austintexas.gov/resource/e3qf-htd9.csv')
aus_20 = pd.read_csv('https://data.austintexas.gov/resource/vc9m-ha4y.csv')

In [4]:
# Concatenating the datasets
aus_final = pd.concat([aus_17, aus_18, aus_19, aus_20])

C:\Users\Robert\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [5]:
# Checking null values
display(aus_final.isnull().sum())
display("-------------------------------")
# Displaying column index
display(aus_final.columns)

bias                             0
date_of_incident                36
date_of_incident_day_of_week    19
day_of_week                     36
incident_number                  0
month                            0
notes                           53
number_of_offenders_over        36
number_of_offenders_over_18     19
number_of_offenders_under       36
number_of_offenders_under_18    19
number_of_victims_over_18        0
number_of_victims_under_18      36
number_of_vitims_under_18       19
offense                         38
offense_location                 0
offense_s                       17
race_ethnic_of_offender_s       36
race_ethnicity_of_offenders     36
race_or_ethnic_of_offender      38
victim_type                     19
dtype: int64

'-------------------------------'

Index(['bias', 'date_of_incident', 'date_of_incident_day_of_week',
       'day_of_week', 'incident_number', 'month', 'notes',
       'number_of_offenders_over', 'number_of_offenders_over_18',
       'number_of_offenders_under', 'number_of_offenders_under_18',
       'number_of_victims_over_18', 'number_of_victims_under_18',
       'number_of_vitims_under_18', 'offense', 'offense_location', 'offense_s',
       'race_ethnic_of_offender_s', 'race_ethnicity_of_offenders',
       'race_or_ethnic_of_offender', 'victim_type'],
      dtype='object')

In [6]:
# Creating a copy of all the object columns
cat_aus_final = aus_final.select_dtypes('object').copy()
display(cat_aus_final.head(10))
display('----------------------------------')
display(aus_final.isnull().sum())
display('----------------------------------')
display(cat_aus_final.isnull().values.sum())

# Taking care of nan rows...remember we're just playing around for now
cat_aus_final = cat_aus_final.fillna(cat_aus_final['date_of_incident'].value_counts().index[0])

display('----------------------------------')
display(cat_aus_final.isnull().values.sum())
display('----------------------------------')
display(cat_aus_final['bias'].value_counts().count())
display('----------------------------------')
display(cat_aus_final.head())
display('----------------------------------')
display(cat_aus_final.tail())

,bias,date_of_incident,date_of_incident_day_of_week,day_of_week,incident_number,month,notes,offense,offense_location,offense_s,race_ethnic_of_offender_s,race_ethnicity_of_offenders,race_or_ethnic_of_offender,victim_type
0,Anti-Black or African American,NaN,01/01/2017/Sun,NaN,2017-241137,January,NaN,Aggravated Assault,Park/Playground,NaN,NaN,NaN,White/Not Hispanic,Individual
1,Anti-White,NaN,02/01/2017/Wed,NaN,2017-580344,February,NaN,Aggravated Assault,Highway/Road/Alley/Street/Sidewalk,NaN,NaN,NaN,Black or African American/Not Hispanic,Individual
2,Anti-Jewish,NaN,03/21/2017/Tues,NaN,2017-800291,March,NaN,Destruction,Highway/Road/Alley/Street/Sidewalk,NaN,NaN,NaN,Unknown,Other
3,Anti-Jewish,NaN,04/12/2017/Wed,NaN,2017-1021534,April,NaN,Simple Assault,Air/Bus/Train Terminal,NaN,NaN,NaN,White/Unknown,Individual
4,Anti-Gay (Male),NaN,05/15/2017/Mon,NaN,2017-1351550,May,NaN,Simple Assault,Residence/Home,NaN,NaN,NaN,White/Not Hispanic,Individual
5,Anti-Gay (Male),NaN,06/04/2017/Sun,NaN,2017-1550219,June,NaN,Simple Assault,Highway/Road/Alley/Street/Sidewalk,NaN,NaN,NaN,White/Unknown,Individual
6,Anti-Hispanic or Latino,NaN,6/15/2017/Thurs,NaN,2017-1660914,June,NaN,Intimidation,Bank/Savings and Loan,NaN,NaN,NaN,White/Not Hispanic,Individual
7,Anti-Black or African American,NaN,06/17/2017/Sat,NaN,2017-1681488,June,NaN,Simple Assault,Highway/Road/Alley/Street/Sidewalk,NaN,NaN,NaN,White/Not Hispanic,Individual
8,Anti-Black or African American,NaN,07/01/2017/Sat,NaN,2017-1820275,July,NaN,Simple Assault,Highway/Road/Alley/Street/Sidewalk,NaN,NaN,NaN,Group of Multiple Races,Individual
9,Anti-Black or African American,NaN,07/05/2017/Wed,NaN,2017-1862526,July,NaN,Intimidation,Residence/Home,NaN,NaN,NaN,Group of Multiple Races,Individual


'----------------------------------'

bias                             0
date_of_incident                36
date_of_incident_day_of_week    19
day_of_week                     36
incident_number                  0
month                            0
notes                           53
number_of_offenders_over        36
number_of_offenders_over_18     19
number_of_offenders_under       36
number_of_offenders_under_18    19
number_of_victims_over_18        0
number_of_victims_under_18      36
number_of_vitims_under_18       19
offense                         38
offense_location                 0
offense_s                       17
race_ethnic_of_offender_s       36
race_ethnicity_of_offenders     36
race_or_ethnic_of_offender      38
victim_type                     19
dtype: int64

'----------------------------------'

328

...

18

'----------------------------------'

,bias,date_of_incident,date_of_incident_day_of_week,day_of_week,incident_number,month,notes,offense,offense_location,offense_s,race_ethnic_of_offender_s,race_ethnicity_of_offenders,race_or_ethnic_of_offender,victim_type
0,Anti-Black or African American,2020-05-20T00:00:00.000,01/01/2017/Sun,2020-05-20T00:00:00.000,2017-241137,January,2020-05-20T00:00:00.000,Aggravated Assault,Park/Playground,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,White/Not Hispanic,Individual
1,Anti-White,2020-05-20T00:00:00.000,02/01/2017/Wed,2020-05-20T00:00:00.000,2017-580344,February,2020-05-20T00:00:00.000,Aggravated Assault,Highway/Road/Alley/Street/Sidewalk,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Black or African American/Not Hispanic,Individual
2,Anti-Jewish,2020-05-20T00:00:00.000,03/21/2017/Tues,2020-05-20T00:00:00.000,2017-800291,March,2020-05-20T00:00:00.000,Destruction,Highway/Road/Alley/Street/Sidewalk,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Unknown,Other
3,Anti-Jewish,2020-05-20T00:00:00.000,04/12/2017/Wed,2020-05-20T00:00:00.000,2017-1021534,April,2020-05-20T00:00:00.000,Simple Assault,Air/Bus/Train Terminal,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,White/Unknown,Individual
4,Anti-Gay (Male),2020-05-20T00:00:00.000,05/15/2017/Mon,2020-05-20T00:00:00.000,2017-1351550,May,2020-05-20T00:00:00.000,Simple Assault,Residence/Home,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,White/Not Hispanic,Individual


'----------------------------------'

,bias,date_of_incident,date_of_incident_day_of_week,day_of_week,incident_number,month,notes,offense,offense_location,offense_s,race_ethnic_of_offender_s,race_ethnicity_of_offenders,race_or_ethnic_of_offender,victim_type
2,Anti-Gay (Male); Anti-Jewish,2020-03-22T00:00:00.000,2020-05-20T00:00:00.000,Sunday,2020-5011788,March,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Residence/Home,Criminal Mischief,2020-05-20T00:00:00.000,Unknown,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000
3,Anti-Buddhist,2020-04-20T00:00:00.000,2020-05-20T00:00:00.000,Monday,2020-5015689,April,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Church/Synagogue/Temple/Mosque,Criminal Mischief,2020-05-20T00:00:00.000,Unknown,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000
4,Anti-Gay (Male); Anti-Transgender,2020-04-29T00:00:00.000,2020-05-20T00:00:00.000,Wednesday,2020-5016804,April,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Department/Discount Store,Assault by Threat,2020-05-20T00:00:00.000,Black/Non-Hispanic,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000
5,Anti-Black or African American,2020-05-17T00:00:00.000,2020-05-20T00:00:00.000,Sunday,2020-1381131,May,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Convenience Store,Assault by Contact,2020-05-20T00:00:00.000,White/Non-Hispanic,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000
6,Anti-Hispanic or Latino,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Wednesday,2020-1410411,May,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000,Streets/Highway/Road/Alley,Assault with Injury,2020-05-20T00:00:00.000,White/Non-Hispanic,2020-05-20T00:00:00.000,2020-05-20T00:00:00.000


In [7]:
# Saving the above copy
cat_aus_final.to_csv(r"C:\Users\Robert\OneDrive\Desktop\cat_aus_final.csv")

In [8]:
# Removing messy columns 
aus_final.drop(['date_of_incident', 'date_of_incident_day_of_week', 'day_of_week', 'notes', 'number_of_offenders_over',
                'number_of_offenders_over_18', 'number_of_offenders_under', 'number_of_offenders_under_18', 
                'number_of_victims_under_18', 'number_of_vitims_under_18', 'offense', 'offense_s', 'race_ethnic_of_offender_s', 
                'race_ethnicity_of_offenders', 'race_or_ethnic_of_offender', 'victim_type'], axis=1, inplace=True)

# Displaying clean dataset head & tail
display(aus_final.head())
display('----------------------------------')
display(aus_final.tail())
display('----------------------------------')
# Rechecking for null values
display(aus_final.isnull().sum())

,bias,incident_number,month,number_of_victims_over_18,offense_location
0,Anti-Black or African American,2017-241137,January,1,Park/Playground
1,Anti-White,2017-580344,February,1,Highway/Road/Alley/Street/Sidewalk
2,Anti-Jewish,2017-800291,March,0,Highway/Road/Alley/Street/Sidewalk
3,Anti-Jewish,2017-1021534,April,0,Air/Bus/Train Terminal
4,Anti-Gay (Male),2017-1351550,May,0,Residence/Home


'----------------------------------'

,bias,incident_number,month,number_of_victims_over_18,offense_location
2,Anti-Gay (Male); Anti-Jewish,2020-5011788,March,1,Residence/Home
3,Anti-Buddhist,2020-5015689,April,1,Church/Synagogue/Temple/Mosque
4,Anti-Gay (Male); Anti-Transgender,2020-5016804,April,1,Department/Discount Store
5,Anti-Black or African American,2020-1381131,May,1,Convenience Store
6,Anti-Hispanic or Latino,2020-1410411,May,1,Streets/Highway/Road/Alley


'----------------------------------'

bias                         0
incident_number              0
month                        0
number_of_victims_over_18    0
offense_location             0
dtype: int64

We can still go a bit farther in the cleaning process:

1. The 'incident_number' column can be split along the '-' -- we can name a new 'year' column and convert it into datetime, and we can create a new 'incident_number' column. 
2. There are various descriptions in the 'bias' column that can be categorized into one variable as 'anti-lgbt.' Let's see what we can do with these. 
3. Also, we can convert the 'bias' column into a category type.

In [9]:
# Converting 'bias' & 'offense_location' columns to categorical data
aus_final['bias'] = aus_final['bias'].astype('category')
aus_final['offense_location'] = aus_final['offense_location'].astype('category')

display(aus_final.head())
display('----------------------------------')
display(aus_final.dtypes)

,bias,incident_number,month,number_of_victims_over_18,offense_location
0,Anti-Black or African American,2017-241137,January,1,Park/Playground
1,Anti-White,2017-580344,February,1,Highway/Road/Alley/Street/Sidewalk
2,Anti-Jewish,2017-800291,March,0,Highway/Road/Alley/Street/Sidewalk
3,Anti-Jewish,2017-1021534,April,0,Air/Bus/Train Terminal
4,Anti-Gay (Male),2017-1351550,May,0,Residence/Home


'----------------------------------'

bias                         category
incident_number                object
month                          object
number_of_victims_over_18       int64
offense_location             category
dtype: object

What I want to do now is split the 'incident_number' column along the '-' because the #s before the '-' clearly indicate the year the incident takes place, which I want to merge with the corresponding months in the 'month' column, and keep the numbers after the '-' as the 'incident_number.' 

In [10]:
# It took me a few tries but I think I finally got it figured out! Now for the final push! Let's pray everybody! :P 
new = aus_final["incident_number"].str.split("-", n = 1, expand = True)
aus_final["year"]= new[0]
aus_final["occurence_number"]= new[1]
aus_final.drop(columns =["incident_number"], inplace = True)
aus_final['date'] = aus_final[['month', 'year']].agg('-'.join, axis=1)
aus_final.drop(['month', 'occurence_number', 'year'], axis=1, inplace=True)
aus_final = aus_final[['date', 'bias', 'number_of_victims_over_18', 'offense_location']]
aus_final.rename(columns={'number_of_victims_over_18': 'victims'}, inplace=True)
aus_final['date'] = pd.to_datetime(aus_final['date'])
aus_final.set_index('date', inplace=True)

# Showing the final product
display(aus_final.head())
display('----------------------------------')
display(aus_final.tail())
display('----------------------------------')
display(aus_final.info())
display('----------------------------------')
display(aus_final.columns)
display('----------------------------------')
display(aus_final.index)

,bias,victims,offense_location
date,,,
2017-01-01,Anti-Black or African American,1,Park/Playground
2017-02-01,Anti-White,1,Highway/Road/Alley/Street/Sidewalk
2017-03-01,Anti-Jewish,0,Highway/Road/Alley/Street/Sidewalk
2017-04-01,Anti-Jewish,0,Air/Bus/Train Terminal
2017-05-01,Anti-Gay (Male),0,Residence/Home


'----------------------------------'

,bias,victims,offense_location
date,,,
2020-03-01,Anti-Gay (Male); Anti-Jewish,1,Residence/Home
2020-04-01,Anti-Buddhist,1,Church/Synagogue/Temple/Mosque
2020-04-01,Anti-Gay (Male); Anti-Transgender,1,Department/Discount Store
2020-05-01,Anti-Black or African American,1,Convenience Store
2020-05-01,Anti-Hispanic or Latino,1,Streets/Highway/Road/Alley


'----------------------------------'

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 55 entries, 2017-01-01 to 2020-05-01
Data columns (total 3 columns):
bias                55 non-null category
victims             55 non-null int64
offense_location    55 non-null category
dtypes: category(2), int64(1)
memory usage: 2.5 KB


None

'----------------------------------'

Index(['bias', 'victims', 'offense_location'], dtype='object')

'----------------------------------'

DatetimeIndex(['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-06-01', '2017-06-01',
               '2017-07-01', '2017-07-01', '2017-07-01', '2017-07-01',
               '2017-10-01', '2017-10-01', '2017-11-01', '2017-11-01',
               '2017-11-01', '2018-01-01', '2018-01-01', '2018-02-01',
               '2018-02-01', '2018-03-01', '2018-03-01', '2018-03-01',
               '2018-04-01', '2018-04-01', '2018-05-01', '2018-06-01',
               '2018-06-01', '2018-08-01', '2018-08-01', '2018-09-01',
               '2018-09-01', '2018-10-01', '2018-10-01', '2018-12-01',
               '2019-01-01', '2019-01-01', '2019-02-01', '2019-03-01',
               '2019-04-01', '2019-06-01', '2019-08-01', '2019-08-01',
               '2019-08-01', '2019-11-01', '2019-12-01', '2019-12-01',
               '2020-03-01', '2020-03-01', '2020-03-01', '2020-04-01',
               '2020-04-01', '2020-05-01', '2020-05-01'],
              dtype

## Exploratory Analysis

In [11]:
# Creating a variable for the biases
bias_count = aus_final['bias'].value_counts()

display(bias_count)
display('----------------------------------')
display(bias_count.sum())

Anti-Black or African American       10
Anti-Gay (Male)                      10
Anti-Black                            8
Anti-White                            3
Anti-Jewish                           3
Anti-Hispanic or Latino               3
Anti-Hispanic                         3
Anti-Transgender                      2
Anti-Gay                              2
Anti-Gay (Male); Anti-Transgender     2
Anti-Islamic (Muslim)                 2
Anti-Gay (Male); Anti-Jewish          1
Anti-Hispanic/Latino                  1
Anti-Disability                       1
Anti-Buddhist                         1
Anti-Islamic(Muslim)                  1
Anti-Lesbian                          1
Anti-Arab                             1
Name: bias, dtype: int64

'----------------------------------'

55

### What total percentage do anti-lgbt oriented hate crimes make of the total? 

In [12]:
aus_final.to_csv(r"C:\Users\Robert\OneDrive\Desktop\aus_final.csv")